In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"01988134-1fae-7002-9617-6e4c877f9720\"")

In [ ]:
hex_project_name = _hex_json.loads("\"LMOB CLI - Skuld Senior Cut\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# import jinja2
# raw_query = """
#     ---run time 1.5 minute
#     USE WAREHOUSE TEMP_CREDIT_RISK;
#     
#     --- select accept offer, unsecured accounts
#     select
#         CLI.*
#         ,SNR.skuld_snr AS SKULD_SNR_SCORE
#         ,PMMO.*
#         ,coalesce(HRAM.HRAM_FLAG,False) AS FLAG_HRAM
#         ,'Bucket' || floor(skuld_snr*10) as SKULD_SNR_SCORE_BUCKET
#         ,DATEDIFF(month, OFFER_DATE, TO_DATE(STMT_MTH || '-01', 'YYYY-MM-DD')) AS CLI_STMT
#     FROM
#     (
#         SELECT
#         accountid
#         ,offer_date
#         ,accepted_flag
#         from
#         SANDBOX.MARKWANG.VW_CLI_
#         where 1=1
#         and subproductname = 'UNSECURED'
#         and CLI_MOB > 6
#     ) CLI
#     ---attach skuld senior score, on the day of CLI offer
#     LEFT JOIN 
#     (
#         SELECT 
#         ACCOUNTID
#         ,prediction as skuld_snr
#         ,REFERENCEDATE
#         FROM 
#         EXTERNAL_INGESTION.BACKFILLS.RISK_SKULD_SNR_V2_1_BACKFILL_2022_10_01_TO_2025_01_01
#         -- ADVANCED_ANALYTICS.BATCH.RISK_SKULD_SNR
#     )SNR
#     ON CLI.ACCOUNTID = SNR.ACCOUNTID
#     AND CLI.OFFER_DATE = DATE(SNR.REFERENCEDATE)
#     ---attach performance data
#     LEFT JOIN 
#     (
#         SELECT
#         ACCOUNTID as ACCOUNTID_PMMO
#         ,VINTAGE
#         ,STMT_NUM
#         ,STMT_MTH
#         ,CLI_ACCEPTED_FLAG
#         ,earl_originalbrand as BRAND
#         ,AM_TU_BAND
#         ,BFC - COF - GUCO as RAIM
#         FROM 
#         SANDBOX.MARKWANG.PM_MONITORING 
#         where 1=1
#         and secure_type = 'UNSECURED'
#     )PMMO
#     ON CLI.ACCOUNTID = PMMO.ACCOUNTID_PMMO
#     --- flag HRAM
#     LEFT JOIN
#     (
#         SELECT
#         ACCOUNTID
#         , ID_DATE as HRAM_DATE
#         , True as HRAM_FLAG
#         FROM
#         SANDBOX.MARKWANG.HRAM_ACCTS
#     ) HRAM
#     ON CLI.ACCOUNTID = HRAM.ACCOUNTID
#     WHERE 1=1
#     and SKULD_SNR_SCORE is not NULL
#     and BRAND in ('NEO','SIENNA')
#     ;
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
## Set Vintage Range
vintage_start ='2023-07'
vintage_end ='2024-06'

In [ ]:
# Filter to selected vintage only
CLI_GROUP_SELECT = CLI_GROUP[
    (CLI_GROUP['OFFER_DATE'].astype(str).str[:7] >= vintage_start) &
    (CLI_GROUP['OFFER_DATE'].astype(str).str[:7] <= vintage_end)
]


### Performance Analysis - 1yr RAIM trend, aggregated and per record



In [ ]:
# Filter to responder only
CLI_VIN_PERFORMANCE = CLI_GROUP_SELECT[CLI_GROUP_SELECT['ACCEPTED_FLAG'] == 1].copy()
# Filter to 1-12 stmt only
CLI_VIN_PERFORMANCE = CLI_VIN_PERFORMANCE[CLI_VIN_PERFORMANCE['CLI_STMT'].between(1, 12)]
# Add raim_cuml
CLI_VIN_PERFORMANCE_1YR = CLI_VIN_PERFORMANCE.groupby(['BRAND','SKULD_SNR_SCORE_BUCKET','CLI_STMT'])['RAIM'].sum().reset_index(drop=False)
CLI_VIN_PERFORMANCE_1YR['RAIM_CUML'] = CLI_VIN_PERFORMANCE_1YR.groupby(['BRAND','SKULD_SNR_SCORE_BUCKET'])['RAIM'].cumsum()

In [ ]:
import altair
chart_CLI_VIN_PERFORMANCE_1YR = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "series data layer",
                    "name": "series_01989f85-9044-7aad-8bef-e8bdf21d5dc1",
                    "layer": [
                        {
                            "description": "line series layer",
                            "transform": [
                                {
                                    "joinaggregate": [
                                        {
                                            "field": "RAIM_CUML",
                                            "op": "sum",
                                            "as": "sum_y_by_x"
                                        }
                                    ],
                                    "groupby": [
                                        "CLI_STMT"
                                    ]
                                },
                                {
                                    "filter": "isValid(datum[\"CLI_STMT\"])"
                                }
                            ],
                            "layer": [
                                {
                                    "description": "line mark layer",
                                    "mark": {
                                        "type": "line",
                                        "point": false,
                                        "clip": true,
                                        "strokeCap": "square",
                                        "strokeJoin": "round",
                                        "cursor": "crosshair"
                                    },
                                    "params": [
                                        {
                                            "name": "interval_intervalselection_0",
                                            "select": {
                                                "type": "interval",
                                                "encodings": [
                                                    "x"
                                                ],
                                                "zoom": false,
                                                "mark": {
                                                    "fill": "#669EFF",
                                                    "fillOpacity": 0.07,
                                                    "stroke": "#669EFF",
                                                    "strokeOpacity": 0.4
                                                }
                                            }
                                        },
                                        {
                                            "name": "legend_pointselection_0",
                                            "select": {
                                                "type": "point",
                                                "encodings": [
                                                    "color"
                                                ]
                                            },
                                            "bind": "legend"
                                        },
                                        {
                                            "name": "legend_pointhover_0",
                                            "select": {
                                                "type": "point",
                                                "encodings": [
                                                    "color"
                                                ]
                                            },
                                            "bind": "legend"
                                        }
                                    ],
                                    "encoding": {
                                        "opacity": {
                                            "value": 1
                                        }
                                    }
                                },
                                {
                                    "description": "line point mark layer",
                                    "mark": {
                                        "type": "point",
                                        "filled": true,
                                        "clip": true
                                    },
                                    "encoding": {
                                        "size": {
                                            "value": 50
                                        },
                                        "opacity": {
                                            "condition": {
                                                "test": {
                                                    "or": [
                                                        {
                                                            "param": "legend_pointhover_0",
                                                            "empty": false
                                                        }
                                                    ],
                                                    "empty": false
                                                },
                                                "value": 1
                                            },
                                            "value": 0
                                        }
                                    }
                                },
                                {
                                    "description": "pivot tooltip point mark layer",
                                    "mark": {
                                        "type": "point",
                                        "tooltip": true,
                                        "clip": true
                                    },
                                    "encoding": {
                                        "opacity": {
                                            "value": 0,
                                            "condition": {
                                                "param": "pivot_hover_01989f85-9044-7aad-8bef-e8bdf21d5dc1",
                                                "value": 1,
                                                "empty": false
                                            }
                                        },
                                        "size": {
                                            "value": 80
                                        },
                                        "tooltip": [
                                            {
                                                "field": "CLI_STMT",
                                                "type": "quantitative",
                                                "formatType": "NUMBER_FORMATTER",
                                                "format": {
                                                    "format": "NUMBER",
                                                    "columnType": "NUMBER",
                                                    "numDecimalDigits": -1,
                                                    "currency": "USD",
                                                    "nanFormat": ""
                                                },
                                                "title": "CLI_STMT"
                                            },
                                            {
                                                "field": "RAIM_CUML",
                                                "type": "quantitative",
                                                "aggregate": "sum",
                                                "formatType": "NUMBER_FORMATTER",
                                                "format": {
                                                    "format": "NUMBER",
                                                    "columnType": "NUMBER",
                                                    "numDecimalDigits": -1,
                                                    "currency": "USD",
                                                    "nanFormat": ""
                                                },
                                                "title": "Sum of RAIM_CUML"
                                            },
                                            {
                                                "field": "SKULD_SNR_SCORE_BUCKET",
                                                "type": "ordinal",
                                                "title": "SKULD_SNR_SCORE_BUCKET"
                                            }
                                        ]
                                    },
                                    "params": [
                                        {
                                            "name": "pivot_hover_01989f85-9044-7aad-8bef-e8bdf21d5dc1",
                                            "select": {
                                                "type": "point",
                                                "on": "mouseover",
                                                "clear": "mouseout",
                                                "nearest": true,
                                                "fields": [
                                                    "CLI_STMT",
                                                    "SKULD_SNR_SCORE_BUCKET"
                                                ]
                                            }
                                        }
                                    ]
                                }
                            ],
                            "encoding": {
                                "x": {
                                    "field": "CLI_STMT",
                                    "type": "quantitative",
                                    "title": "CLI_STMT",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "USD",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    }
                                },
                                "y": {
                                    "field": "RAIM_CUML",
                                    "type": "quantitative",
                                    "aggregate": "sum",
                                    "title": "Sum of RAIM_CUML",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "USD",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    }
                                },
                                "color": {
                                    "field": "SKULD_SNR_SCORE_BUCKET",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#F58518",
                                            "#E45756",
                                            "#72B7B2",
                                            "#54A24B",
                                            "#EECA3B",
                                            "#B279A2",
                                            "#FF9DA6",
                                            "#9D755D",
                                            "#BAB0AC"
                                        ]
                                    },
                                    "sort": {
                                        "field": "SKULD_SNR_SCORE_BUCKET",
                                        "order": "ascending"
                                    },
                                    "legend": {
                                        "symbolOpacity": 1,
                                        "symbolType": "stroke"
                                    },
                                    "title": "SKULD_SNR_SCORE_BUCKET"
                                }
                            }
                        }
                    ]
                }
            ],
            "transform": []
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {
            "interval_intervalselection_0": {
                "type": "interval",
                "datetimeFields": [],
                "derived": []
            },
            "legend_pointselection_0": {
                "type": "point",
                "datetimeFields": [],
                "derived": []
            }
        },
        "columnNameMappings": {
            "CHART_FOLD_KEYS": "Y-Axis Field",
            "CHART_FOLD_VALUES": "Values"
        }
    }
}
""")
chart_CLI_VIN_PERFORMANCE_1YR.datasets.layer00 = CLI_VIN_PERFORMANCE_1YR.to_json(orient='records')
chart_CLI_VIN_PERFORMANCE_1YR.display(actions=False)

In [ ]:
CLI_VIN_PERFORMANCE_1YR_AVG = CLI_VIN_PERFORMANCE.groupby(['BRAND','SKULD_SNR_SCORE_BUCKET','CLI_STMT'])['RAIM'].mean().reset_index(drop=False)
CLI_VIN_PERFORMANCE_1YR_AVG['RAIM_CUML'] = CLI_VIN_PERFORMANCE_1YR_AVG.groupby(['BRAND','SKULD_SNR_SCORE_BUCKET'])['RAIM'].cumsum()

In [ ]:
import altair
chart_CLI_VIN_PERFORMANCE_1YR_AVG = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "series data layer",
                    "name": "series_01989f92-d3fa-7aad-8bf6-9f1134b4d3f3",
                    "layer": [
                        {
                            "description": "line series layer",
                            "transform": [
                                {
                                    "joinaggregate": [
                                        {
                                            "field": "RAIM_CUML",
                                            "op": "sum",
                                            "as": "sum_y_by_x"
                                        }
                                    ],
                                    "groupby": [
                                        "CLI_STMT"
                                    ]
                                },
                                {
                                    "filter": "isValid(datum[\"CLI_STMT\"])"
                                }
                            ],
                            "layer": [
                                {
                                    "description": "line mark layer",
                                    "mark": {
                                        "type": "line",
                                        "point": false,
                                        "clip": true,
                                        "strokeCap": "square",
                                        "strokeJoin": "round",
                                        "cursor": "crosshair"
                                    },
                                    "params": [
                                        {
                                            "name": "interval_intervalselection_0",
                                            "select": {
                                                "type": "interval",
                                                "encodings": [
                                                    "x"
                                                ],
                                                "zoom": false,
                                                "mark": {
                                                    "fill": "#669EFF",
                                                    "fillOpacity": 0.07,
                                                    "stroke": "#669EFF",
                                                    "strokeOpacity": 0.4
                                                }
                                            }
                                        },
                                        {
                                            "name": "legend_pointselection_0",
                                            "select": {
                                                "type": "point",
                                                "encodings": [
                                                    "color"
                                                ]
                                            },
                                            "bind": "legend"
                                        },
                                        {
                                            "name": "legend_pointhover_0",
                                            "select": {
                                                "type": "point",
                                                "encodings": [
                                                    "color"
                                                ]
                                            },
                                            "bind": "legend"
                                        }
                                    ],
                                    "encoding": {
                                        "opacity": {
                                            "value": 1
                                        }
                                    }
                                },
                                {
                                    "description": "line point mark layer",
                                    "mark": {
                                        "type": "point",
                                        "filled": true,
                                        "clip": true
                                    },
                                    "encoding": {
                                        "size": {
                                            "value": 50
                                        },
                                        "opacity": {
                                            "condition": {
                                                "test": {
                                                    "or": [
                                                        {
                                                            "param": "legend_pointhover_0",
                                                            "empty": false
                                                        }
                                                    ],
                                                    "empty": false
                                                },
                                                "value": 1
                                            },
                                            "value": 0
                                        }
                                    }
                                },
                                {
                                    "description": "pivot tooltip point mark layer",
                                    "mark": {
                                        "type": "point",
                                        "tooltip": true,
                                        "clip": true
                                    },
                                    "encoding": {
                                        "opacity": {
                                            "value": 0,
                                            "condition": {
                                                "param": "pivot_hover_01989f92-d3fa-7aad-8bf6-9f1134b4d3f3",
                                                "value": 1,
                                                "empty": false
                                            }
                                        },
                                        "size": {
                                            "value": 80
                                        },
                                        "tooltip": [
                                            {
                                                "field": "CLI_STMT",
                                                "type": "quantitative",
                                                "formatType": "NUMBER_FORMATTER",
                                                "format": {
                                                    "format": "NUMBER",
                                                    "columnType": "NUMBER",
                                                    "numDecimalDigits": -1,
                                                    "currency": "USD",
                                                    "nanFormat": ""
                                                },
                                                "title": "CLI_STMT"
                                            },
                                            {
                                                "field": "RAIM_CUML",
                                                "type": "quantitative",
                                                "aggregate": "sum",
                                                "formatType": "NUMBER_FORMATTER",
                                                "format": {
                                                    "format": "NUMBER",
                                                    "columnType": "NUMBER",
                                                    "numDecimalDigits": -1,
                                                    "currency": "USD",
                                                    "nanFormat": ""
                                                },
                                                "title": "Sum of RAIM_CUML"
                                            },
                                            {
                                                "field": "SKULD_SNR_SCORE_BUCKET",
                                                "type": "ordinal",
                                                "title": "SKULD_SNR_SCORE_BUCKET"
                                            }
                                        ]
                                    },
                                    "params": [
                                        {
                                            "name": "pivot_hover_01989f92-d3fa-7aad-8bf6-9f1134b4d3f3",
                                            "select": {
                                                "type": "point",
                                                "on": "mouseover",
                                                "clear": "mouseout",
                                                "nearest": true,
                                                "fields": [
                                                    "CLI_STMT",
                                                    "SKULD_SNR_SCORE_BUCKET"
                                                ]
                                            }
                                        }
                                    ]
                                }
                            ],
                            "encoding": {
                                "x": {
                                    "field": "CLI_STMT",
                                    "type": "quantitative",
                                    "title": "CLI_STMT",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "USD",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    }
                                },
                                "y": {
                                    "field": "RAIM_CUML",
                                    "type": "quantitative",
                                    "aggregate": "sum",
                                    "title": "Sum of RAIM_CUML",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "USD",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    }
                                },
                                "color": {
                                    "field": "SKULD_SNR_SCORE_BUCKET",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#F58518",
                                            "#E45756",
                                            "#72B7B2",
                                            "#54A24B",
                                            "#EECA3B",
                                            "#B279A2",
                                            "#FF9DA6",
                                            "#9D755D",
                                            "#BAB0AC"
                                        ]
                                    },
                                    "sort": {
                                        "field": "SKULD_SNR_SCORE_BUCKET",
                                        "order": "ascending"
                                    },
                                    "legend": {
                                        "symbolOpacity": 1,
                                        "symbolType": "stroke"
                                    },
                                    "title": "SKULD_SNR_SCORE_BUCKET"
                                }
                            }
                        }
                    ]
                }
            ],
            "transform": []
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {
            "interval_intervalselection_0": {
                "type": "interval",
                "datetimeFields": [],
                "derived": []
            },
            "legend_pointselection_0": {
                "type": "point",
                "datetimeFields": [],
                "derived": []
            }
        },
        "columnNameMappings": {
            "CHART_FOLD_KEYS": "Y-Axis Field",
            "CHART_FOLD_VALUES": "Values"
        }
    }
}
""")
chart_CLI_VIN_PERFORMANCE_1YR_AVG.datasets.layer00 = CLI_VIN_PERFORMANCE_1YR_AVG.to_json(orient='records')
chart_CLI_VIN_PERFORMANCE_1YR_AVG.display(actions=False)

### Incremental Value - responder vs. non-responder, per record



In [ ]:
# Filter brand, statement
CLI_GROUP_RESPOND = CLI_GROUP_SELECT[CLI_GROUP_SELECT['BRAND'] == 'NEO'].copy()
CLI_GROUP_RESPOND = CLI_GROUP_RESPOND[CLI_GROUP_RESPOND['CLI_STMT'].between(1, 12)]
# Aggregate to statement level
CLI_GROUP_RESPOND_RAIM_STMT = CLI_GROUP_RESPOND.groupby(['ACCEPTED_FLAG','SKULD_SNR_SCORE_BUCKET','CLI_STMT']).agg({'RAIM': 'mean','ACCOUNTID':'count'}).reset_index(drop=False)



In [ ]:
CLI_GROUP_RESPOND_RAIM_COMPARE = CLI_GROUP_RESPOND_RAIM_STMT[CLI_GROUP_RESPOND_RAIM_STMT['ACCEPTED_FLAG'] == 0][['SKULD_SNR_SCORE_BUCKET', 'CLI_STMT', 'RAIM','ACCOUNTID']]
CLI_GROUP_RESPOND_RAIM_COMPARE = CLI_GROUP_RESPOND_RAIM_COMPARE.merge(
    CLI_GROUP_RESPOND_RAIM_STMT[CLI_GROUP_RESPOND_RAIM_STMT['ACCEPTED_FLAG'] == 1][['SKULD_SNR_SCORE_BUCKET', 'CLI_STMT', 'RAIM','ACCOUNTID']],
    on=['SKULD_SNR_SCORE_BUCKET', 'CLI_STMT'],
    suffixes=('_nonresponder', '_responder')
)
CLI_GROUP_RESPOND_RAIM_COMPARE = CLI_GROUP_RESPOND_RAIM_COMPARE.groupby('SKULD_SNR_SCORE_BUCKET').agg({'RAIM_nonresponder':'sum', 'ACCOUNTID_nonresponder':'max','RAIM_responder':'sum', 'ACCOUNTID_responder':'max'}).reset_index(drop=False)
CLI_GROUP_RESPOND_RAIM_COMPARE['DIFF'] = CLI_GROUP_RESPOND_RAIM_COMPARE['RAIM_responder'] - CLI_GROUP_RESPOND_RAIM_COMPARE['RAIM_nonresponder']
CLI_GROUP_RESPOND_RAIM_COMPARE

,SKULD_SNR_SCORE_BUCKET,RAIM_nonresponder,ACCOUNTID_nonresponder,RAIM_responder,ACCOUNTID_responder,DIFF
0,Bucket0,14.113907,40251,56.305653,33828,42.191746
1,Bucket1,223.566609,1982,294.871738,4968,71.305129
2,Bucket2,233.979369,729,314.599283,2557,80.619914
3,Bucket3,227.353281,328,220.138068,1333,-7.215213
4,Bucket4,206.885840,137,195.057074,832,-11.828766
5,Bucket5,336.594948,85,175.293716,470,-161.301232
6,Bucket6,-124.296617,51,-109.256952,277,15.039665
7,Bucket7,473.012510,14,-606.674454,127,-1079.686965
8,Bucket8,-1880.067993,5,-731.404970,40,1148.663023
9,Bucket9,-2234.593281,7,-559.726803,15,1674.866478


In [ ]:
CLI_GROUP_RESPOND_RAIM_TREND = CLI_GROUP_RESPOND_RAIM_STMT.copy()
CLI_GROUP_RESPOND_RAIM_TREND['RAIM_CUML'] = CLI_GROUP_RESPOND_RAIM_STMT.groupby(['ACCEPTED_FLAG','SKULD_SNR_SCORE_BUCKET'])['RAIM'].cumsum()

In [ ]:

from plotly.subplots import make_subplots
# write plotly code to generate line chart of CLIP_GROUP_REPOND_RAIM_TREND
buckets = CLI_GROUP_RESPOND_RAIM_TREND['SKULD_SNR_SCORE_BUCKET'].unique()
num_buckets = len(buckets)
cols = 3
rows = (num_buckets + cols - 1) // cols  

# Create subplots
fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f"{bucket}" for bucket in buckets])

for bucket in buckets:
    i = list(buckets).index(bucket)
    row = (i // cols) + 1
    col = (i % cols) + 1
    data_subset = CLI_GROUP_RESPOND_RAIM_TREND[CLI_GROUP_RESPOND_RAIM_TREND['SKULD_SNR_SCORE_BUCKET'] == bucket]
    px_fig = px.line(
        data_subset,
        x="CLI_STMT",
        y="RAIM_CUML",
        color="ACCEPTED_FLAG",
        color_discrete_map={0:'blue',1:'orange'}
    )
    
    for trace in px_fig.data:
        trace.update(legendgroup=trace.name, name=trace.name)
        fig.add_trace(trace, row=row, col=col)

# Update layout
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',width=900, height=600)
names = set()
fig.for_each_trace(
    lambda trace: trace.update(showlegend=False) if (trace.name in names) else names.add(trace.name))

fig.show()

### Account Distribution - non-HRAM/HRAM accts over Skuld cut



In [ ]:
## profile analysis

# ?WHY NO SKULD SCORE
# Filter to responder
CLI_GROUP_W_SKULD = CLI_GROUP_SELECT[CLI_GROUP_SELECT['ACCEPTED_FLAG'] == 1].sort_values(by=['OFFER_DATE','ACCOUNTID'])
# Treat each accepted CLI offer as single record
CLI_GROUP_W_SKULD_OFFERDAY = CLI_GROUP_W_SKULD.query('CLI_STMT == 1', engine='python')
# Filter brand
CLI_GROUP_W_SKULD_OFFERDAY = CLI_GROUP_W_SKULD_OFFERDAY[CLI_GROUP_W_SKULD_OFFERDAY['BRAND'] == 'NEO']


In [ ]:
## Find the right skuld level and cut most non hram % under and most hram % above
def get_skuld_cut_points(df,cut_range = (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)):
    df_result = pd.DataFrame(columns=['cut_level', 'non_hram_cut_percentage', 'non_hrm_cut_volume', 'hram_cut_percentage', 'hram_cut_volume'])
    for cut_level in cut_range:
        non_hram_cut_total = df.query('FLAG_HRAM == False and SKULD_SNR_SCORE >= @cut_level')['ACCOUNTID'].count()
        non_hram_cut_percentage = non_hram_cut_total/df.query('FLAG_HRAM == False')['ACCOUNTID'].count()
        hram_cut_total = df.query('FLAG_HRAM == True and SKULD_SNR_SCORE >= @cut_level')['ACCOUNTID'].count()
        hram_cut_percentage = hram_cut_total/df.query('FLAG_HRAM == True')['ACCOUNTID'].count()
        df_result.loc[len(df_result)] = [cut_level, non_hram_cut_percentage, non_hram_cut_total, hram_cut_percentage, hram_cut_total]
    return df_result


In [ ]:
df_result = get_skuld_cut_points(CLI_GROUP_W_SKULD_OFFERDAY)
df_result

,cut_level,non_hram_cut_percentage,non_hrm_cut_volume,hram_cut_percentage,hram_cut_volume
0,0.1,0.235995,10397.0,0.570694,222.0
1,0.2,0.125136,5513.0,0.354756,138.0
2,0.3,0.068231,3006.0,0.226221,88.0
3,0.4,0.038655,1703.0,0.149100,58.0
4,0.5,0.020156,888.0,0.105398,41.0
5,0.6,0.010010,441.0,0.046272,18.0
6,0.7,0.003972,175.0,0.017995,7.0
7,0.8,0.001180,52.0,0.007712,3.0
8,0.9,0.000318,14.0,0.002571,1.0


In [ ]:
px.histogram(CLI_GROUP_W_SKULD_OFFERDAY.query('FLAG_HRAM == False').sort_values(by='SKULD_SNR_SCORE_BUCKET'), x='SKULD_SNR_SCORE_BUCKET')


In [ ]:
px.histogram(CLI_GROUP_W_SKULD_OFFERDAY.query('FLAG_HRAM == True').sort_values(by='SKULD_SNR_SCORE_BUCKET'), x='SKULD_SNR_SCORE_BUCKET')

### Multi CLI accts profile



In [ ]:
CLI_GROUP_TOTAL = CLI_GROUP_SELECT[
    (CLI_GROUP_SELECT['ACCEPTED_FLAG'] == 1)
    &(CLI_GROUP_SELECT['BRAND'] == 'NEO')
    &(CLI_GROUP_SELECT['CLI_STMT'].between(1, 12))
    ].sort_values(by=['ACCOUNTID', 'OFFER_DATE'], ignore_index=True)

CLI_GROUP_MULTI = CLI_GROUP_TOTAL.groupby(['ACCOUNTID', 'OFFER_DATE'])['ACCEPTED_FLAG'].mean().reset_index(drop=False)
CLI_GROUP_MULTI['ACCEPTED_FLAG_CUML'] = CLI_GROUP_MULTI.groupby('ACCOUNTID')['ACCEPTED_FLAG'].cumsum()
CLI_GROUP_MULTI_CUML = CLI_GROUP_TOTAL.merge(CLI_GROUP_MULTI[['ACCOUNTID', 'OFFER_DATE','ACCEPTED_FLAG_CUML']], on=['ACCOUNTID', 'OFFER_DATE'], how='left')
CLI_GROUP_MULTI_CUML['MULTI_FLAG'] = CLI_GROUP_MULTI_CUML['ACCEPTED_FLAG_CUML']>1
CLI_GROUP_MULTI_CUML['OFFER_VINTAGE'] = CLI_GROUP_MULTI_CUML['OFFER_DATE'].astype(str).str[:7]

In [ ]:
CLI_GROUP_MULTI_COUNT = CLI_GROUP_MULTI.groupby('ACCOUNTID')['ACCEPTED_FLAG'].sum().reset_index()
CLI_GROUP_MULTI_COUNT.groupby('ACCEPTED_FLAG').count().reset_index()

,ACCEPTED_FLAG,ACCOUNTID
0,1.0,24429
1,2.0,9066
2,3.0,624
3,4.0,3


In [ ]:
## Avg Skuld score

CLI_GROUP_MULTI_CUML_COMP = CLI_GROUP_MULTI_CUML[CLI_GROUP_MULTI_CUML['CLI_STMT'] == 1].groupby(['OFFER_VINTAGE', 'MULTI_FLAG']).agg({'SKULD_SNR_SCORE':'mean'}).reset_index()
CLI_GROUP_MULTI_COMP = pd.merge(
    CLI_GROUP_MULTI_CUML_COMP.query('MULTI_FLAG == True',engine = 'python')[['OFFER_VINTAGE','SKULD_SNR_SCORE']],
    CLI_GROUP_MULTI_CUML_COMP.query('MULTI_FLAG == False',engine = 'python')[['OFFER_VINTAGE','SKULD_SNR_SCORE']],
    on=["OFFER_VINTAGE"],
    how="left",
    suffixes=( '_multi','_singular')
)

CLI_GROUP_MULTI_COMP

,OFFER_VINTAGE,SKULD_SNR_SCORE_multi,SKULD_SNR_SCORE_singular
0,2023-11,0.052216,0.066615
1,2023-12,0.081772,0.099598
2,2024-01,0.077041,0.086613
3,2024-02,0.092223,0.088234
4,2024-03,0.062565,0.074537
5,2024-04,0.067644,0.094202
6,2024-05,0.062311,0.081566
7,2024-06,0.064589,0.082556


In [ ]:
## sum 1 yr RAIM
CLI_GROUP_MULTI_CUML_RAIM = CLI_GROUP_MULTI_CUML.groupby(['OFFER_VINTAGE','MULTI_FLAG','CLI_STMT']).agg({'RAIM': 'mean'}).reset_index()
CLI_GROUP_MULTI_CUML_RAIM = CLI_GROUP_MULTI_CUML_RAIM.groupby(['OFFER_VINTAGE','MULTI_FLAG']).agg({'RAIM': 'sum'}).reset_index()
CLI_GROUP_MULTI_COMP_RAIM = pd.merge(
    CLI_GROUP_MULTI_CUML_RAIM.query('MULTI_FLAG == True',engine = 'python')[['OFFER_VINTAGE','RAIM']],
    CLI_GROUP_MULTI_CUML_RAIM.query('MULTI_FLAG == False',engine = 'python')[['OFFER_VINTAGE','RAIM']],
    on=["OFFER_VINTAGE"],
    how="left",
    suffixes=( '_multi','_singular')
)

CLI_GROUP_MULTI_COMP_RAIM

,OFFER_VINTAGE,RAIM_multi,RAIM_singular
0,2023-11,194.402280,96.264834
1,2023-12,148.183191,105.919746
2,2024-01,141.892175,79.102887
3,2024-02,181.097563,135.265970
4,2024-03,159.529377,86.158367
5,2024-04,81.276817,83.015347
6,2024-05,94.443975,61.379767
7,2024-06,83.911512,94.255283
